## 2016 play by play data

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.cm as cm
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy
import zipfile
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import seaborn as sn
from sklearn.metrics import silhouette_samples, silhouette_score


sys.setrecursionlimit(100000)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

### import 2017 nhl data set

In [2]:
da = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_pbp20172018.csv')
#da = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_pbp20172018.csv')
da = da.drop('Unnamed: 0', axis=1)

- rename columns and create a season variable

In [3]:
da['Season'] = 2017
da = da.rename(columns={ 'Game_Id' : 'GameNumber', 'Ev_Zone': 'EventZone', 'Time_Elapsed': 'Time', 'Ev_Team': 'EventTeamCode', 'Description': 'EventDetail', 'Away_Team': 'ATeamCode', 'Home_Team':'HTeamCode', 'p1_name': 'EventPlayer1','p2_name': 'EventPlayer2', 'p3_name': 'EventPlayer3', 'awayPlayer1': 'APlayer1', 'awayPlayer2': 'APlayer2', 'awayPlayer3': 'APlayer3', 'awayPlayer4': 'APlayer4', 'awayPlayer5': 'APlayer5', 'awayPlayer6': 'APlayer6', 'homePlayer1': 'HPlayer1', 'homePlayer2': 'HPlayer2', 'homePlayer3': 'HPlayer3','homePlayer4': 'HPlayer4', 'homePlayer5': 'HPlayer5','homePlayer6': 'HPlayer6', 'Away_Score': 'AGoals', 'Home_Score': 'HGoals', 'Away_Goalie': 'AGoalie', 'Home_Goalie': 'HGoalie', 'Away_Players': 'PlayersA', 'Home_Players': 'PlayersH' })
da = da[['Season', 'GameNumber', 'Date', 'Period', 'Time', 'Strength', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'ATeamCode', 'APlayer1', 'APlayer2', 'APlayer3', 'APlayer4', 'APlayer5', 'APlayer6', 'AGoalie', 'PlayersA', 'HTeamCode', 'HPlayer1', 'HPlayer2', 'HPlayer3', 'HPlayer4', 'HPlayer5', 'HPlayer6', 'HGoalie', 'PlayersH']]
da = da.sort_values(['Season', 'GameNumber', 'Period', 'Time'], ascending=[True, True, True, True])

- keep only regular season games and exclude irrelevant on-ice events

In [4]:
da = da[da['GameNumber'] <= 21271]
da = da[da['Period'] <= 4]
da = da[da['Period'] >= 1]
da = da[da['Event']!='STOP']
da = da[da['Event']!='EISTR']
da = da[da['Event']!='EIEND']
da = da[da['Event']!='PSTR']
da = da[da['Event']!='PEND']
da = da[da['Event']!='SOC']
da = da[da['Event']!='GEND']

- create an event number variable that will count the number of events per game. 
- create an advantage type variable for even strength, power play and short handed situations.

In [5]:
da['EventNumber'] = da.groupby(['Season', 'GameNumber']).cumcount()+1

In [6]:
da['AdvantageType'] = da.apply(lambda x: 'EV' if ((x['PlayersA'] == x['PlayersH']) & (x['PlayersA'] != 1) & (x['PlayersH'] != 1)) else 'PP' if ((x['PlayersA'] > x['PlayersH']) & (x['EventTeamCode'] == x['ATeamCode'])) else 'PP' if ((x['PlayersA'] < x['PlayersH']) & (x['EventTeamCode'] == x['HTeamCode'])) else 'SH' if ((x['PlayersA'] < x['PlayersH']) & (x['EventTeamCode'] == x['ATeamCode'])) else 'SH' if ((x['PlayersA'] > x['PlayersH']) & (x['EventTeamCode'] == x['HTeamCode'])) else 'PP' if ((x['PlayersA'] == 1 ) & (x['PlayersH'] == 1)) else np.nan, axis=1) 

In [7]:
da['AdvantageType'].value_counts()

EV    287925
PP     34368
SH     15327
Name: AdvantageType, dtype: int64

- reshape data from wide to long for away team players and home team players

In [8]:
db = da.copy()
a = [col for col in db.columns if 'APlayer' in col]
b = [col for col in db.columns if 'HPlayer' in col]
db = pd.lreshape(db, {'APlayer' : a, 'HPlayer' : b })
db.head()

,AGoalie,ATeamCode,AdvantageType,Date,Event,EventDetail,EventNumber,EventPlayer1,EventPlayer2,EventPlayer3,EventTeamCode,GameNumber,HGoalie,HTeamCode,Period,PlayersA,PlayersH,Season,Strength,Time,APlayer,HPlayer
0,FREDERIK ANDERSEN,TOR,EV,2017-10-04,FAC,TOR won Neu. Zone - TOR #43 KADRI vs WPG #55 S...,4,NAZEM KADRI,MARK SCHEIFELE,NaN,TOR,20001,STEVE MASON,WPG,1,6,6,2017,5x5,0:00,PATRICK MARLEAU,MARK SCHEIFELE
1,FREDERIK ANDERSEN,TOR,EV,2017-10-04,BLOCK,"TOR #43 KADRI BLOCKED BY WPG #39 ENSTROM, Wri...",5,TOBIAS ENSTROM,NAZEM KADRI,NaN,TOR,20001,STEVE MASON,WPG,1,6,6,2017,5x5,0:12,PATRICK MARLEAU,MARK SCHEIFELE
2,FREDERIK ANDERSEN,TOR,EV,2017-10-04,SHOT,"WPG ONGOAL - #44 MORRISSEY, Wrist, Off. Zone, ...",6,JOSH MORRISSEY,NaN,NaN,WPG,20001,STEVE MASON,WPG,1,6,6,2017,5x5,0:38,PATRICK MARLEAU,SHAWN MATTHIAS
3,FREDERIK ANDERSEN,TOR,EV,2017-10-04,HIT,"TOR #47 KOMAROV HIT WPG #44 MORRISSEY, Def. Zone",7,LEO KOMAROV,JOSH MORRISSEY,NaN,TOR,20001,STEVE MASON,WPG,1,6,6,2017,5x5,0:41,TYLER BOZAK,SHAWN MATTHIAS
4,FREDERIK ANDERSEN,TOR,EV,2017-10-04,SHOT,"WPG ONGOAL - #16 MATTHIAS, Wrist, Off. Zone, 1...",8,SHAWN MATTHIAS,NaN,NaN,WPG,20001,STEVE MASON,WPG,1,6,6,2017,5x5,0:49,TYLER BOZAK,SHAWN MATTHIAS


In [9]:
db = db[['Season', 'GameNumber', 'Date', 'Period', 'Time', 'AdvantageType', 'Strength', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'ATeamCode', 'APlayer', 'AGoalie', 'PlayersA', 'HTeamCode', 'HPlayer', 'HGoalie', 'PlayersH']]
db = db.sort_values(['Season', 'GameNumber', 'Period', 'Time'], ascending=[True, True, True, True])

- reshape data from wide to long for away and home team code and player respectfully.

In [10]:
dc = db.copy()
dc = dc.rename(columns={'EventTeamCode': 'EventTeam', 'EventPlayer1': 'EventP1', 'EventPlayer2': 'EventP2', 'EventPlayer3': 'EventP3', 'PlayersA':'PA', 'PlayersH': 'PH'})

In [11]:
a = [col for col in dc.columns if 'Code' in col]
b = [col for col in dc.columns if 'Player' in col]
c = [col for col in dc.columns if 'Goalie' in col]
d = [col for col in dc.columns if 'Position' in col]
dc = pd.lreshape(dc, {'TeamCode' : a, 'PlayerName' : b, 'Goalie': c})

In [12]:
dc = dc.rename(columns={'EventTeam': 'EventTeamCode', 'EventP1': 'EventPlayer1', 'EventP2': 'EventPlayer2', 'EventP3': 'EventPlayer3', 'PA':'PlayersA', 'PH': 'PlayersH'})

In [13]:
dc.shape

(3838847, 19)

### games played

- calculate the games each player participated in for the duration of the 2017 regular season.

In [14]:
s = dc.copy()
s = s[['Season', 'GameNumber', 'TeamCode', 'PlayerName']]
s = s.sort_values(['Season', 'GameNumber', 'TeamCode'], ascending=[True, True, True])
s = s.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])

In [15]:
s['GP'] = s.groupby(['Season', 'TeamCode','PlayerName'])['GameNumber'].transform('count')
s = s.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
s = s[['Season', 'TeamCode', 'PlayerName', 'GP']]
s.head()

,Season,TeamCode,PlayerName,GP
0,2017,TOR,PATRICK MARLEAU,82
1,2017,TOR,NAZEM KADRI,80
2,2017,TOR,LEO KOMAROV,74
3,2017,TOR,NIKITA ZAITSEV,60
4,2017,TOR,JAKE GARDINER,82


In [16]:
s.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
dtype: int64

In [17]:
s.shape

(1067, 4)

### shots for and against

- calculate the shots against each player faced througout the duration of a season. 
- create shot for and shot against variables. If event team code is indiferrent to team code assign a value of 1. If not, assign a value of 0. 
- **goals are a result of a shot and therefore need to be added to a player's shot total**. Display total shots per player for a season.

In [19]:
sa = dc.copy()
sa = sa[sa['Event'].isin(['SHOT','GOAL'])]

In [20]:
sa['ShotF'] = sa.apply(lambda x: 1 if x['EventTeamCode'] == x['TeamCode'] else 0, axis=1)
sa['ShotsF'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['ShotF'].transform('sum')

sa['ShotA'] = sa.apply(lambda x: 1 if x['EventTeamCode'] != x['TeamCode'] else 0, axis=1)
sa['ShotsA'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['ShotA'].transform('sum')

- create shot for and shot against variables for even strength situations only. If event team code is indiferrent to team code assign a value of 1. If not, assign a value of 0. 
- **goals are a result of a shot and therefore need to be added to a player's shot total**. Display total shots per player for a season.

In [21]:
sa['EVShotF'] = sa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] == x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] == x['TeamCode'])) else 0, axis=1)
sa['EVShotsF'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVShotF'].transform('sum')

sa['EVShotA'] = sa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] != x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] != x['TeamCode'])) else 0, axis=1)
sa['EVShotsA'] = sa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVShotA'].transform('sum')

- calculate the shot differential for all shots and even strength shots.

In [22]:
sa['DShots'] = sa['ShotsF'] - sa['ShotsA']
sa['EVDShots'] = sa['EVShotsF'] - sa['EVShotsA']

In [23]:
sa = sa [['Season', 'TeamCode', 'PlayerName', 'ShotsF', 'ShotsA', 'DShots', 'EVShotsF', 'EVShotsA', 'EVDShots' ]]
sa = sa.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
sa.head()

,Season,TeamCode,PlayerName,ShotsF,ShotsA,DShots,EVShotsF,EVShotsA,EVDShots
12,2017,TOR,PATRICK MARLEAU,749,730,19,607,703,-96
13,2017,TOR,NAZEM KADRI,811,714,97,597,712,-115
14,2017,TOR,LEO KOMAROV,508,710,-202,463,525,-62
15,2017,TOR,NIKITA ZAITSEV,556,827,-271,549,655,-106
16,2017,TOR,JAKE GARDINER,965,935,30,806,902,-96


In [24]:
sa.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
ShotsF        0
ShotsA        0
DShots        0
EVShotsF      0
EVShotsA      0
EVDShots      0
dtype: int64

In [25]:
sa.shape

(1067, 9)

### goals for and against

- create goals for and goals against variable. If event team code is the same as team code assign a value of 1 for goals for and a value of 0 for goals against. If event team code is different to team code, assign a value of 0 for golas for and a value of 1 for goals against. Display total goals for and against per player for a season.

In [26]:
gfa = dc.copy()
gfa = gfa[gfa['Event'] == 'GOAL']

In [27]:
gfa['EVGoalF'] = gfa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] == x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] == x['TeamCode'])) else 0, axis=1)
gfa['EVGoalA'] = gfa.apply(lambda x: 1 if ((x['AdvantageType'] == 'EV') & (x['EventTeamCode'] != x['TeamCode'])) else 1 if ((x['AdvantageType'] == 'SH') & (x['EventTeamCode'] != x['TeamCode'])) else 0, axis=1)

In [28]:
gfa['GoalF'] = gfa.apply(lambda x: 1 if (x['EventTeamCode'] == x['TeamCode']) else 0, axis=1)
gfa['GoalA'] = gfa.apply(lambda x: 1 if (x['EventTeamCode'] != x['TeamCode']) else 0, axis=1)

In [29]:
gfa['EVGoalsF'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVGoalF'].transform('sum')
gfa['EVGoalsA'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['EVGoalA'].transform('sum')
gfa['GoalsF'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['GoalF'].transform('sum')
gfa['GoalsA'] = gfa.groupby(['Season', 'TeamCode', 'PlayerName'])['GoalA'].transform('sum')

In [30]:
gfa = gfa[['Season', 'GameNumber', 'TeamCode', 'PlayerName' , 'GoalsF', 'GoalsA', 'EVGoalsF', 'EVGoalsA']]
gfa = gfa.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
gfa['Plus/Minus'] = gfa['EVGoalsF'] - gfa['EVGoalsA']
gfa = gfa.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
gfa = gfa[['Season', 'TeamCode', 'PlayerName' , 'GoalsF', 'GoalsA', 'EVGoalsF', 'EVGoalsA', 'Plus/Minus']]
gfa.head()

,Season,TeamCode,PlayerName,GoalsF,GoalsA,EVGoalsF,EVGoalsA,Plus/Minus
159,2017,TOR,MITCHELL MARNER,86,53,50,51,-1
161,2017,TOR,TYLER BOZAK,80,42,45,42,3
163,2017,TOR,NAZEM KADRI,88,52,54,52,2
165,2017,TOR,JAMES VAN RIEMSDYK,82,47,44,47,-3
167,2017,TOR,MORGAN RIELLY,97,78,66,65,1


In [31]:
gfa.shape

(1036, 8)

### create dataframes for all on-ice event types

#### goals dataframe (dg)

In [32]:
dg = dc.copy()
dg = dg[dg['Event'] == 'GOAL']

In [33]:
dg = dg[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dg = dg.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a goal variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total goals per player for a season.

In [34]:
dg['Goal'] = dg.apply(lambda x: 1 if (x['Event'] == 'GOAL') else 0, axis=1)
dg = dg.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dg['Goals'] = dg.groupby(['Season','TeamCode', 'PlayerName'])['Goal'].transform('sum')

- keep one observation per player by game and drop duplicates. This purpose of this step is to calculate the quantity of games per player for a season. 

In [35]:
dg = dg.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dg = dg.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dg = dg[['Season', 'TeamCode', 'PlayerName', 'Goals']]
dg.head()

,Season,TeamCode,PlayerName,Goals
159,2017,TOR,NAZEM KADRI,32
236,2017,TOR,JAMES VAN RIEMSDYK,36
254,2017,TOR,WILLIAM NYLANDER,20
974,2017,TOR,PATRICK MARLEAU,27
1078,2017,WPG,MARK SCHEIFELE,23


In [36]:
dg.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Goals         0
dtype: int64

In [37]:
dg.shape

(772, 4)

#### assists dataframe (dast)

In [38]:
dast = dc.copy()
dast = dast[dast['Event'] == 'GOAL']

In [39]:
dast = dast[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer2', 'EventPlayer3']]
dast = dast.rename(columns={'EventPlayer2': 'Assist1Name', 'EventPlayer3': 'Assist2Name', 'EventTeamCode':'TeamCode'})

- reshape assist players 1 and 2 to create one column of assist players

In [40]:
a = [col for col in dast.columns if 'Name' in col]
dast = pd.lreshape(dast, {'PlayerName' : a})

- create an assist variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total goals per player for a season.

In [41]:
dast['Assist'] = dast.apply(lambda x: 1 if (x['Event'] == 'GOAL') else 0, axis=1)
dast = dast.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dast['Assists'] = dast.groupby(['Season','TeamCode', 'PlayerName'])['Assist'].transform('sum')

- keep one observation per player by game and drop duplicates. This purpose of this step is to calculate the quantity of games per player for a season. 

In [42]:
dast = dast.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dast = dast.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dast = dast[['Season', 'TeamCode', 'PlayerName', 'Assists']]
dast.head()

,Season,TeamCode,PlayerName,Assists
0,2017,TOR,JAMES VAN RIEMSDYK,18
5,2017,TOR,TYLER BOZAK,32
11,2017,TOR,JAKE GARDINER,47
17,2017,TOR,AUSTON MATTHEWS,29
23,2017,TOR,NAZEM KADRI,23


In [43]:
dast.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Assists       0
dtype: int64

In [44]:
dast.shape

(872, 4)

#### shot dataframe (ds)

In [45]:
ds = dc.copy()
ds = ds[ds['Event'] == 'SHOT']

In [46]:
ds = ds[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
ds = ds.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})
ds.head()

,Season,GameNumber,EventNumber,Event,TeamCode,PlayerName
12,2017,20001,6,SHOT,WPG,JOSH MORRISSEY
13,2017,20001,6,SHOT,WPG,JOSH MORRISSEY
14,2017,20001,6,SHOT,WPG,JOSH MORRISSEY
15,2017,20001,6,SHOT,WPG,JOSH MORRISSEY
16,2017,20001,6,SHOT,WPG,JOSH MORRISSEY


- create a shot variable. If event type is a goal assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [47]:
ds['Shot'] = ds.apply(lambda x: 1 if (x['Event'] == 'SHOT') else 0, axis=1)
ds = ds.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
ds['Shots'] = ds.groupby(['Season','TeamCode', 'PlayerName'])['Shot'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [48]:
ds = ds.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
ds = ds.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
ds = ds[['Season', 'TeamCode', 'PlayerName', 'Shots']]
ds.head()

,Season,TeamCode,PlayerName,Shots
12,2017,WPG,JOSH MORRISSEY,93
24,2017,WPG,SHAWN MATTHIAS,25
36,2017,WPG,DUSTIN BYFUGLIEN,185
58,2017,WPG,ADAM LOWRY,62
78,2017,WPG,BRYAN LITTLE,113


In [49]:
ds.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Shots         0
dtype: int64

In [50]:
ds.shape

(947, 4)

#### block dataframe (db)

In [51]:
dbl = dc.copy()
dbl = dbl[dbl['Event'] == 'BLOCK']

In [52]:
dbl.shape

(428943, 19)

In [53]:
dbl['EventPlayer2'].isnull().sum()

0

In [54]:
dbl['EventPlayer3'].isnull().sum()

428943

In [55]:
dbl['BlockTeam'] = dbl.EventDetail.str.split(' ', expand = True)[6]
dbl['BlockTeamCode'] = dbl.BlockTeam.str.split(' ', expand = True)[0]

In [56]:
dbl = dbl[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventPlayer1', 'BlockTeamCode']]
dbl = dbl.rename(columns={'EventPlayer1': 'PlayerName', 'BlockTeamCode':'TeamCode'})

- create a block variable. If event type is a block assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [57]:
dbl['Block'] = dbl.apply(lambda x: 1 if (x['Event'] == 'BLOCK') else 0, axis=1)
dbl = dbl.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dbl['Blocks'] = dbl.groupby(['Season','TeamCode', 'PlayerName'])['Block'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [58]:
dbl = dbl.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dbl = dbl.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dbl = dbl[['Season', 'TeamCode', 'PlayerName', 'Blocks']]
dbl.head()

,Season,TeamCode,PlayerName,Blocks
6,2017,WPG,TOBIAS ENSTROM,50
63,2017,TOR,LEO KOMAROV,45
68,2017,TOR,NIKITA ZAITSEV,134
110,2017,WPG,TYLER MYERS,115
272,2017,TOR,NAZEM KADRI,26


In [59]:
dbl.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Blocks        0
dtype: int64

In [60]:
dbl.shape

(1152, 4)

#### hit dataframe (dh)

In [61]:
dhit = dc.copy()
dhit = dhit[dhit['Event'] == 'HIT']

In [62]:
dhit.shape

(648631, 19)

In [63]:
dhit['EventPlayer2'].isnull().sum()

0

In [64]:
dhit['EventPlayer3'].isnull().sum()

648631

In [65]:
dhit = dhit[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dhit = dhit.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a hit variable. If event type is a hit assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [66]:
dhit['Hit'] = dhit.apply(lambda x: 1 if (x['Event'] == 'HIT') else 0, axis=1)
dhit = dhit.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dhit['Hits'] = dhit.groupby(['Season','TeamCode', 'PlayerName'])['Hit'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [67]:
dhit = dhit.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dhit = dhit.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dhit = dhit[['Season', 'TeamCode', 'PlayerName', 'Hits']]
dhit.head()

,Season,TeamCode,PlayerName,Hits
18,2017,TOR,LEO KOMAROV,219
104,2017,WPG,JACOB TROUBA,84
200,2017,TOR,RON HAINSEY,91
206,2017,TOR,MATT MARTIN,146
266,2017,WPG,DMITRY KULIKOV,69


In [68]:
dhit.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Hits          0
dtype: int64

In [69]:
dhit.shape

(962, 4)

#### penalty dataframe (dpen)

In [70]:
dp = dc.copy()
dp = dp[dp['Event'] == 'PENL']

- use event detail to find the duration of a given penalty and to assign it to the proper player. Major penalty is 5 minutes, so a value of 5 is assigned for every event that had a major penalty.

In [71]:
dp['Pen'] = dp.EventDetail.str.split(')', expand = True)[0]
dp['Penal'] = dp.Pen.str.split('(', expand = True)[1]
dp['Penalt'] = dp.Penal.str.split(' ', expand = True)[0]
dp['Penalty'] = dp.apply(lambda x: 5 if x['Penalt'] == 'maj' else x['Penalt'], axis=1)
dp['Penalty'] = dp['Penalty'].convert_objects(convert_numeric=True)
dp = dp[['Season', 'GameNumber', 'Period', 'Time', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'TeamCode', 'PlayerName', 'Penalty']]

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


- use event detail to find penalty team code and penalty player name.

In [72]:
dp['PenaltyTeamCode'] = dp.EventDetail.str.split('\\s', expand = True)[0]
dp['PN'] = dp.EventDetail.str.split('\\s', expand = True)[1]
dp['PNumber'] = dp.PN.str.split('#', expand = True)[1]
dp['PenaltyPlayerNumber'] = dp.PNumber.str.split(' ', expand = True)[0]
dp['PenaltyPlayerLName'] = dp.EventDetail.str.split('\\s', expand = True)[2]
#dp['PenaltyPlayerLName'] = dp.PPlLN.str.split(' ', expand = True)[0]
dp = dp[['Season', 'GameNumber', 'Period', 'Time', 'EventNumber', 'Event', 'EventDetail', 'EventTeamCode', 'EventPlayer1', 'EventPlayer2', 'EventPlayer3', 'TeamCode', 'PlayerName', 'Penalty', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyPlayerLName']]

- seperate player name into first and last. The purpose is to connect the correct player name to penalty player name.

In [73]:
dp['PlayerFName'] = dp.PlayerName.str.split('\\s', expand = True)[0]
dp['PlayerLName'] = dp.PlayerName.str.split('\\s', expand = True)[1]
dp['PenaltyPlayerFName'] = dp.apply(lambda x: x['PlayerFName'] if ((x['PenaltyTeamCode'] == x['TeamCode']) & (x['PenaltyPlayerNumber'] == x['PenaltyPlayerNumber']) & (x['PlayerLName'] == x['PenaltyPlayerLName'])) else np.nan, axis=1)
dp['PenaltyPlayerFName'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyPlayerLName'])['PenaltyPlayerFName'].apply(lambda x: x.ffill().bfill())

In [74]:
dp['PenaltyName'] = dp.apply(lambda x: x['PlayerName'] if ((x['PenaltyTeamCode'] == x['TeamCode']) & (x['PenaltyPlayerNumber'] == x['PenaltyPlayerNumber']) & (x['PlayerLName'] == x['PenaltyPlayerLName']) & (x['PlayerFName'] == x['PenaltyPlayerFName'])) else np.nan, axis=1)
dp['PenaltyName'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerLName', 'PenaltyPlayerNumber', 'PenaltyPlayerLName'])['PenaltyName'].apply(lambda x: x.ffill().bfill())


- drop duplicates per game and event number. Calculate the total number of penalties for each player.

In [75]:
dp = dp.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyName'])
dp['Penalties'] = dp.groupby(['Season', 'PenaltyTeamCode', 'PenaltyPlayerNumber', 'PenaltyName'])['Penalty'].transform('sum')
dp['Penalties'] = dp.groupby(['Season','PenaltyTeamCode', 'PenaltyPlayerNumber', 'PlayerName'])['Penalties'].apply(lambda x: x.ffill().bfill())
#dp.head()

- drop duplicates to keep one observation per player for the season.

In [76]:
dp = dp[['Season', 'PenaltyTeamCode', 'PenaltyName', 'Penalties']]
dp = dp.drop_duplicates(['Season', 'PenaltyTeamCode', 'PenaltyName'])
dp = dp.rename(columns={'PenaltyTeamCode':'TeamCode', 'PenaltyName':'PlayerName'})
dp = dp[['Season', 'TeamCode', 'PlayerName', 'Penalties']]
dp.head()

,Season,TeamCode,PlayerName,Penalties
42,2017,TOR,CONNOR BROWN,18.0
128,2017,WPG,JACOB TROUBA,34.0
284,2017,TOR,JAKE GARDINER,32.0
365,2017,TOR,MORGAN RIELLY,14.0
472,2017,TOR,LEO KOMAROV,31.0


- keep observations that are not null. Since a team can be allocated a penalty, those observations are excluded from the data.

In [77]:
dp['PlayerName'].isnull().sum()

31

In [78]:
dp = dp[dp['PlayerName'].notnull()]

In [79]:
dp.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Penalties     0
dtype: int64

In [80]:
dp.shape

(886, 4)

####  miss dataframe (dmiss)

In [81]:
dmiss = dc.copy()
dmiss = dmiss[dmiss['Event'] == 'MISS']

In [82]:
dmiss.shape

(354512, 19)

In [83]:
dmiss['EventPlayer2'].isnull().sum()

354512

In [84]:
dmiss['EventPlayer3'].isnull().sum()

354512

In [85]:
dmiss = dmiss[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dmiss = dmiss.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a miss variable. If event type is a miss assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [86]:
dmiss['Miss'] = dmiss.apply(lambda x: 1 if (x['Event'] == 'MISS') else 0, axis=1)
dmiss = dmiss.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dmiss['Misses'] = dmiss.groupby(['Season','TeamCode', 'PlayerName'])['Miss'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [87]:
dmiss = dmiss.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dmiss = dmiss.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dmiss = dmiss[['Season', 'TeamCode', 'PlayerName', 'Misses']]
dmiss.head()

,Season,TeamCode,PlayerName,Misses
73,2017,WPG,BLAKE WHEELER,81
144,2017,TOR,MITCHELL MARNER,83
212,2017,TOR,CONNOR BROWN,53
329,2017,WPG,TOBIAS ENSTROM,19
347,2017,TOR,RON HAINSEY,43


In [88]:
dmiss.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Misses        0
dtype: int64

In [89]:
dmiss.shape

(915, 4)

####  takeaway dataframe (dtake)

In [90]:
dtake = dc.copy()
dtake = dtake[dtake['Event'] == 'TAKE']

In [91]:
dtake.shape

(214286, 19)

In [92]:
dtake['EventPlayer2'].isnull().sum()

214286

In [93]:
dtake['EventPlayer3'].isnull().sum()

214286

In [94]:
dtake = dtake[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dtake = dtake.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a takeaway variable. If event type is a takeaway assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [95]:
dtake['Takeaway'] = dtake.apply(lambda x: 1 if (x['Event'] == 'TAKE') else 0, axis=1)
dtake = dtake.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dtake['Takeaways'] = dtake.groupby(['Season','TeamCode', 'PlayerName'])['Takeaway'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [96]:
dtake = dtake.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dtake = dtake.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dtake = dtake[['Season', 'TeamCode', 'PlayerName', 'Takeaways']]
dtake.head()

,Season,TeamCode,PlayerName,Takeaways
706,2017,WPG,SHAWN MATTHIAS,7
884,2017,WPG,MARK SCHEIFELE,47
1442,2017,TOR,ERIC FEHR,2
1447,2017,WPG,NIKOLAJ EHLERS,38
1790,2017,PIT,TOM KUHNHACKL,15


In [97]:
dtake.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Takeaways     0
dtype: int64

In [98]:
dtake.shape

(896, 4)

####  giveaway dataframe (dgive)

In [99]:
dgive = dc.copy()
dgive = dgive[dgive['Event'] == 'GIVE']

In [100]:
dgive.shape

(278083, 19)

In [101]:
dgive['EventPlayer2'].isnull().sum()

278083

In [102]:
dgive['EventPlayer3'].isnull().sum()

278083

In [103]:
dgive = dgive[['Season', 'GameNumber', 'EventNumber', 'Event', 'EventTeamCode', 'EventPlayer1']]
dgive = dgive.rename(columns={'EventPlayer1': 'PlayerName', 'EventTeamCode':'TeamCode'})

- create a giveaway variable. If event type is a giveaway assign a value of 1. If not, assign a value of 0. Display total shots per player for a season.

In [104]:
dgive['Giveaway'] = dgive.apply(lambda x: 1 if (x['Event'] == 'GIVE') else 0, axis=1)
dgive = dgive.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName'])
dgive['Giveaways'] = dgive.groupby(['Season','TeamCode', 'PlayerName'])['Giveaway'].transform('sum')

- drop duplicates to keep one observation per player for the season.

In [105]:
dgive = dgive.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])
dgive = dgive.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dgive = dgive[['Season', 'TeamCode', 'PlayerName', 'Giveaways']]
dgive.head()

,Season,TeamCode,PlayerName,Giveaways
53,2017,WPG,DUSTIN BYFUGLIEN,79
182,2017,WPG,PATRIK LAINE,51
359,2017,TOR,RON HAINSEY,85
442,2017,TOR,MITCHELL MARNER,50
498,2017,TOR,JAKE GARDINER,105


In [106]:
dgive.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
Giveaways     0
dtype: int64

In [107]:
dgive.shape

(981, 4)

### import 2017 player shifts

In [108]:
dshift = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_shifts20172018.csv')
#dshift = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_shifts20172018.csv')
dshift = dshift.drop('Unnamed: 0', axis=1)

- rename variables, keep only regular season games and create a season variable.

In [109]:
dshift['Season'] = 2017
dshift= dshift.rename(columns={ 'Game_Id' : 'GameNumber', 'Team': 'TeamCode', 'Player': 'PlayerName' })
dshift = dshift[dshift['GameNumber'] <= 21271]
dshift = dshift[['Season', 'Date', 'GameNumber', 'Period', 'TeamCode', 'PlayerName', 'Player_Id', 'Start', 'End', 'Duration']]
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration
0,2017,2017-10-04,20001,1,WPG,BLAKE WHEELER,8471218,0.0,28.0,28.0
1,2017,2017-10-04,20001,1,WPG,DUSTIN BYFUGLIEN,8470834,0.0,28.0,28.0
2,2017,2017-10-04,20001,1,WPG,TOBIAS ENSTROM,8470828,0.0,28.0,28.0
3,2017,2017-10-04,20001,1,WPG,MARK SCHEIFELE,8476460,0.0,37.0,37.0
4,2017,2017-10-04,20001,1,WPG,MATHIEU PERREAULT,8473618,0.0,37.0,37.0


- calcuate the seconds an individual played per game. Convert seconds to minutes and drop duplicates per season, game number, team and player.

In [110]:
dshift['Seconds'] = dshift.groupby(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])['Duration'].transform('sum')
dshift['Minutes'] = dshift['Seconds']/60
dshift['Minutes'] = dshift['Minutes'].round(2)

In [111]:
dshift = dshift.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'PlayerName'])

In [112]:
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration,Seconds,Minutes
0,2017,2017-10-04,20001,1,WPG,BLAKE WHEELER,8471218,0.0,28.0,28.0,1098.0,18.30
1,2017,2017-10-04,20001,1,WPG,DUSTIN BYFUGLIEN,8470834,0.0,28.0,28.0,1339.0,22.32
2,2017,2017-10-04,20001,1,WPG,TOBIAS ENSTROM,8470828,0.0,28.0,28.0,758.0,12.63
3,2017,2017-10-04,20001,1,WPG,MARK SCHEIFELE,8476460,0.0,37.0,37.0,1142.0,19.03
4,2017,2017-10-04,20001,1,WPG,MATHIEU PERREAULT,8473618,0.0,37.0,37.0,1049.0,17.48


- calcuate the seconds an individual played for the whole regular season. Convert seconds to minutes and drop duplicates per season, game number, team and player.

In [113]:
dshift['TSeconds'] = dshift.groupby(['Season', 'TeamCode', 'PlayerName'])['Seconds'].transform('sum')
dshift['TOI'] = dshift['TSeconds']/60
dshift['TOI'] = dshift['TOI'].round(0)
dshift.head()

,Season,Date,GameNumber,Period,TeamCode,PlayerName,Player_Id,Start,End,Duration,Seconds,Minutes,TSeconds,TOI
0,2017,2017-10-04,20001,1,WPG,BLAKE WHEELER,8471218,0.0,28.0,28.0,1098.0,18.30,98447.0,1641.0
1,2017,2017-10-04,20001,1,WPG,DUSTIN BYFUGLIEN,8470834,0.0,28.0,28.0,1339.0,22.32,100763.0,1679.0
2,2017,2017-10-04,20001,1,WPG,TOBIAS ENSTROM,8470828,0.0,28.0,28.0,758.0,12.63,43941.0,732.0
3,2017,2017-10-04,20001,1,WPG,MARK SCHEIFELE,8476460,0.0,37.0,37.0,1142.0,19.03,74442.0,1241.0
4,2017,2017-10-04,20001,1,WPG,MATHIEU PERREAULT,8473618,0.0,37.0,37.0,1049.0,17.48,60725.0,1012.0


- keep only observation per player for the season

In [114]:
dshift = dshift.drop_duplicates(['Season', 'TeamCode', 'PlayerName'])
dshift = dshift[['Season', 'TeamCode', 'PlayerName', 'TOI']]
dshift.head()

,Season,TeamCode,PlayerName,TOI
0,2017,WPG,BLAKE WHEELER,1641.0
1,2017,WPG,DUSTIN BYFUGLIEN,1679.0
2,2017,WPG,TOBIAS ENSTROM,732.0
3,2017,WPG,MARK SCHEIFELE,1241.0
4,2017,WPG,MATHIEU PERREAULT,1012.0


In [115]:
dshift.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
TOI           0
dtype: int64

In [116]:
dshift.shape

(1067, 4)

### import 2017 player positions

In [117]:
dpos = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/data/nhl_positions20172018.csv')
#dpos = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/data/nhl_positions20172018.csv')

In [118]:
dpos.head()

,Last Name,First Name,Position,TeamCode,FName,LName,PlayerName
0,Hayes,Jimmy,RW,NJD,JIMMY,HAYES,JIMMY HAYES
1,Lehtera,Jori,C,PHI,JORI,LEHTERA,JORI LEHTERA
2,Stone,Michael,D,CGY,MICHAEL,STONE,MICHAEL STONE
3,Hartnell,Scott,LW,NSH,SCOTT,HARTNELL,SCOTT HARTNELL
4,Hainsey,Ron,D,TOR,RON,HAINSEY,RON HAINSEY


In [119]:
dpos['Season'] = 2017
dpos= dpos.rename(columns={ 'Team': 'TeamCode', 'Pos ': 'Position'})
dpos = dpos[['Season', 'TeamCode', 'PlayerName', 'Position']]
dpos.head()

,Season,TeamCode,PlayerName,Position
0,2017,NJD,JIMMY HAYES,RW
1,2017,PHI,JORI LEHTERA,C
2,2017,CGY,MICHAEL STONE,D
3,2017,NSH,SCOTT HARTNELL,LW
4,2017,TOR,RON HAINSEY,D


In [120]:
dpos.dtypes

Season         int64
TeamCode      object
PlayerName    object
Position      object
dtype: object

In [121]:
dpos['Position'] = dpos.apply(lambda x: 'C' if x['Position'] == 'C/LW' else 'C' if x['Position'] == 'C/RW' else 'C' if x['Position'] == 'C/RW/LW' else 'C' if x['Position'] == 'C/LW/RW' else 'W' if x['Position'] == 'RW' else 'W' if x['Position'] == 'LW' else 'W' if x['Position'] == 'LW/C' else 'W' if x['Position'] == 'RW/C' else 'W' if x['Position'] == 'RW/LW' else 'W' if x['Position'] == 'LW/RW' else 'W' if x['Position'] == 'RW/LW/C' else 'W' if x['Position'] == 'RW/C/LW' else 'W' if x['Position'] == 'RW/C/LW' else 'W' if x['Position'] == 'LW/D' else 'W' if x['Position'] == 'LW/C/RW' else 'W' if x['Position'] == 'LW/RW/C' else 'D' if x['Position'] == 'D/LW' else 'D' if x['Position'] == 'D/RW' else 'C' if x['Position'] == 'C/D' else 'W' if x['Position'] == 'RW/D' else x['Position'] , axis=1)
dpos['Position'].value_counts()

W    347
D    306
C    237
G     95
Name: Position, dtype: int64

In [122]:
dpos['TeamCode'] = dpos.apply(lambda x: 'L.A' if x['TeamCode'] == 'LAK' else 'N.J' if x['TeamCode'] == 'NJD' else 'S.J' if x['TeamCode'] == 'SJS' else 'T.B' if x['TeamCode'] == 'TBL' else x['TeamCode'] , axis=1)
dpos.head()

,Season,TeamCode,PlayerName,Position
0,2017,N.J,JIMMY HAYES,W
1,2017,PHI,JORI LEHTERA,C
2,2017,CGY,MICHAEL STONE,D
3,2017,NSH,SCOTT HARTNELL,W
4,2017,TOR,RON HAINSEY,D


In [123]:
dpos.shape

(985, 4)

### merge data frames

- merge games played, goals and assists.

In [124]:
dd = pd.merge(s, dpos, on=['Season', 'TeamCode', 'PlayerName'], how='left').merge(dg,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dast,on=['Season', 'TeamCode', 'PlayerName'], how = 'left')

- forward and backward fill players that have been traded during the season. Fill in positions for three players.

In [126]:
dd['Position'] = dd.groupby(['PlayerName'])['Position'].apply(lambda x: x.ffill().bfill())
dd['Position'] = dd.apply(lambda x: 'C' if x['PlayerName'] == "DANNY O'REGAN" else x['Position'] , axis=1)
dd.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists
0,2017,TOR,PATRICK MARLEAU,82,W,27.0,20.0
1,2017,TOR,NAZEM KADRI,80,C,32.0,23.0
2,2017,TOR,LEO KOMAROV,74,C,7.0,12.0
3,2017,TOR,NIKITA ZAITSEV,60,D,5.0,8.0
4,2017,TOR,JAKE GARDINER,82,D,5.0,47.0


In [128]:
dd.isnull().sum()

Season          0
TeamCode        0
PlayerName      0
GP              0
Position        0
Goals         295
Assists       195
dtype: int64

In [129]:
fix = dd.copy()
fix = fix[fix['Position'].isnull()]
fix.head(20)

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists


- fill in goals and assists with 0 for players who had no points.

In [130]:
dd['Goals'] = dd['Goals'].fillna(0)
dd['Assists'] = dd['Assists'].fillna(0)
dd['Points'] = dd['Goals'] + dd['Assists']
dd.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists,Points
0,2017,TOR,PATRICK MARLEAU,82,W,27.0,20.0,47.0
1,2017,TOR,NAZEM KADRI,80,C,32.0,23.0,55.0
2,2017,TOR,LEO KOMAROV,74,C,7.0,12.0,19.0
3,2017,TOR,NIKITA ZAITSEV,60,D,5.0,8.0,13.0
4,2017,TOR,JAKE GARDINER,82,D,5.0,47.0,52.0


In [131]:
dd.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
Position      0
Goals         0
Assists       0
Points        0
dtype: int64

In [132]:
de = pd.merge(dd, gfa,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(ds,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(sa,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dbl,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dhit,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dp,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dtake,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dgive,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dmiss,on=['Season', 'TeamCode', 'PlayerName'], how = 'left').merge(dshift,on=['Season', 'TeamCode', 'PlayerName'], how = 'left')
de.head()

,Season,TeamCode,PlayerName,GP,Position,Goals,Assists,Points,GoalsF,GoalsA,EVGoalsF,EVGoalsA,Plus/Minus,Shots,ShotsF,ShotsA,DShots,EVShotsF,EVShotsA,EVDShots,Blocks,Hits,Penalties,Takeaways,Giveaways,Misses,TOI
0,2017,TOR,PATRICK MARLEAU,82,W,27.0,20.0,47.0,81.0,59.0,61.0,55.0,6.0,177.0,749,730,19,607,703,-96,26.0,85.0,16.0,28.0,33.0,67.0,1393.0
1,2017,TOR,NAZEM KADRI,80,C,32.0,23.0,55.0,88.0,52.0,54.0,52.0,2.0,178.0,811,714,97,597,712,-115,26.0,93.0,42.0,42.0,48.0,75.0,1341.0
2,2017,TOR,LEO KOMAROV,74,C,7.0,12.0,19.0,42.0,52.0,35.0,35.0,0.0,107.0,508,710,-202,463,525,-62,45.0,219.0,31.0,47.0,42.0,43.0,1168.0
3,2017,TOR,NIKITA ZAITSEV,60,D,5.0,8.0,13.0,60.0,69.0,59.0,51.0,8.0,49.0,556,827,-271,549,655,-106,134.0,113.0,31.0,21.0,76.0,29.0,1333.0
4,2017,TOR,JAKE GARDINER,82,D,5.0,47.0,52.0,97.0,66.0,75.0,63.0,12.0,128.0,965,935,30,806,902,-96,103.0,51.0,32.0,39.0,105.0,68.0,1847.0


- fill in with 0 for all nan values since not all players have been on the ice for a goal for or a goal against. 

In [133]:
de['GoalsF'] = de['GoalsF'].fillna(0)
de['GoalsA'] = de['GoalsA'].fillna(0)
de['EVGoalsF'] = de['EVGoalsF'].fillna(0)
de['EVGoalsA'] = de['EVGoalsA'].fillna(0)
de['Plus/Minus'] = de['Plus/Minus'].fillna(0)
de['Shots'] = de['Shots'].fillna(0)
de['Hits'] = de['Hits'].fillna(0)
de['Blocks'] = de['Blocks'].fillna(0)
de['Penalties'] = de['Penalties'].fillna(0)
de['Giveaways'] = de['Giveaways'].fillna(0)
de['Takeaways'] = de['Takeaways'].fillna(0)
de['Misses'] = de['Misses'].fillna(0)

In [134]:
de.isnull().sum()

Season        0
TeamCode      0
PlayerName    0
GP            0
Position      0
Goals         0
Assists       0
Points        0
GoalsF        0
GoalsA        0
EVGoalsF      0
EVGoalsA      0
Plus/Minus    0
Shots         0
ShotsF        0
ShotsA        0
DShots        0
EVShotsF      0
EVShotsA      0
EVDShots      0
Blocks        0
Hits          0
Penalties     0
Takeaways     0
Giveaways     0
Misses        0
TOI           1
dtype: int64

In [136]:
de = de[de['TOI'].notnull()]

### total player stats for the season

- players that have been traded need to be evaluated for their season performance. Group by sesaon and player name to calcute the sum of each stat. 

In [137]:
de['TGP'] = de.groupby(['Season', 'PlayerName', 'Position'])['GP'].transform('sum')
de['TGoals'] = de.groupby(['Season', 'PlayerName', 'Position'])['Goals'].transform('sum')
de['TAssists'] = de.groupby(['Season', 'PlayerName', 'Position'])['Assists'].transform('sum')
de['TPoints'] = de.groupby(['Season', 'PlayerName', 'Position'])['Points'].transform('sum')
de['TGoalsF'] = de.groupby(['Season', 'PlayerName', 'Position'])['GoalsF'].transform('sum')
de['TGoalsA'] = de.groupby(['Season', 'PlayerName','Position'])['GoalsA'].transform('sum')
de['TEVGoalsF'] = de.groupby(['Season', 'PlayerName', 'Position'])['EVGoalsF'].transform('sum')
de['TEVGoalsA'] = de.groupby(['Season', 'PlayerName', 'Position'])['EVGoalsA'].transform('sum')
de['TPlus/Minus'] = de.groupby(['Season', 'PlayerName', 'Position'])['Plus/Minus'].transform('sum')
de['TShots'] = de.groupby(['Season', 'PlayerName', 'Position'])['Shots'].transform('sum')
de['TShotsF'] = de.groupby(['Season', 'PlayerName', 'Position'])['ShotsF'].transform('sum')
de['TShotsA'] = de.groupby(['Season', 'PlayerName', 'Position'])['ShotsA'].transform('sum')
de['TDShots'] = de.groupby(['Season', 'PlayerName'])['DShots'].transform('sum')
de['TEVShotsF'] = de.groupby(['Season', 'PlayerName', 'Position'])['EVShotsF'].transform('sum')
de['TEVShotsA'] = de.groupby(['Season', 'PlayerName', 'Position'])['EVShotsA'].transform('sum')
de['TEVDShots'] = de.groupby(['Season', 'PlayerName', 'Position'])['EVDShots'].transform('sum')
de['TBlocks'] = de.groupby(['Season', 'PlayerName', 'Position'])['Blocks'].transform('sum')
de['THits'] = de.groupby(['Season', 'PlayerName', 'Position'])['Hits'].transform('sum')
de['TPenalties'] = de.groupby(['Season', 'PlayerName', 'Position'])['Penalties'].transform('sum')
de['TTakeaways'] = de.groupby(['Season', 'PlayerName', 'Position'])['Takeaways'].transform('sum')
de['TGiveaways'] = de.groupby(['Season', 'PlayerName', 'Position'])['Giveaways'].transform('sum')
de['TMisses'] = de.groupby(['Season', 'PlayerName', 'Position'])['Misses'].transform('sum')
de['TTOI'] = de.groupby(['Season', 'PlayerName', 'Position'])['TOI'].transform('sum')

- divide all stats with total time on ice to scale the data for cluster analysis

In [139]:
de['TOIGoals'] = de['TGoals'] / de['TTOI']
de['TOIAssists'] = de['TAssists'] / de['TTOI']
de['TOIPoints'] = de['TPoints'] / de['TTOI']
de['TOIGoalsF'] = de['TGoalsF'] / de['TTOI']
de['TOIGoalsA'] = de['TGoalsA'] / de['TTOI']
de['TOIEVGoalsF'] = de['TEVGoalsF'] / de['TTOI']
de['TOIEVGoalsA'] = de['TEVGoalsA'] / de['TTOI']
de['TOIPlus/Minus'] = de['TPlus/Minus'] / de['TTOI']
de['TOIShots'] = de['TShots'] / de['TTOI']
de['TOIShotsF'] = de['TShotsF'] / de['TTOI']
de['TOIShotsA'] = de['TShotsA'] / de['TTOI']
de['TOIDShots'] = de['TDShots'] / de['TTOI']
de['TOIEVShotsF'] = de['TEVShotsF'] / de['TTOI']
de['TOIEVShotsA'] = de['TEVShotsA'] / de['TTOI']
de['TOIEVDShots'] = de['TEVDShots'] / de['TTOI']
de['TOIBlocks'] = de['TBlocks'] / de['TTOI']
de['TOIHits'] = de['THits'] / de['TTOI']
de['TOIPenalties'] = de['TPenalties'] / de['TTOI']
de['TOITakeaways'] = de['TTakeaways'] / de['TTOI']
de['TOIGiveaways'] = de['TGiveaways'] / de['TTOI']
de['TOIMisses'] = de['TMisses'] / de['TTOI']

- keep sesaon total variables and scaled variables

In [140]:
de = de[['Season', 'TeamCode', 'PlayerName', 'Position', 'TGP', 'TGoals', 'TAssists', 'TPoints', 'TGoalsF', 'TGoalsA', 'TEVGoalsF', 'TEVGoalsA', 'TPlus/Minus', 'TShots', 'TShotsF', 'TShotsA', 'TDShots', 'TEVShotsF', 'TEVShotsA', 'TEVDShots', 'TBlocks', 'THits', 'TPenalties', 'TTakeaways', 'TGiveaways', 'TMisses', 'TTOI', 'TOIGoals', 'TOIAssists', 'TOIPoints', 'TOIGoalsF', 'TOIGoalsA', 'TOIEVGoalsF', 'TOIEVGoalsA', 'TOIPlus/Minus', 'TOIShots', 'TOIShotsF', 'TOIShotsA', 'TOIDShots', 'TOIEVShotsF', 'TOIEVShotsA', 'TOIEVDShots', 'TOIBlocks', 'TOIHits', 'TOIPenalties', 'TOITakeaways', 'TOIGiveaways', 'TOIMisses']]
de.head()

,Season,TeamCode,PlayerName,Position,TGP,TGoals,TAssists,TPoints,TGoalsF,TGoalsA,TEVGoalsF,TEVGoalsA,TPlus/Minus,TShots,TShotsF,TShotsA,TDShots,TEVShotsF,TEVShotsA,TEVDShots,TBlocks,THits,TPenalties,TTakeaways,TGiveaways,TMisses,TTOI,TOIGoals,TOIAssists,TOIPoints,TOIGoalsF,TOIGoalsA,TOIEVGoalsF,TOIEVGoalsA,TOIPlus/Minus,TOIShots,TOIShotsF,TOIShotsA,TOIDShots,TOIEVShotsF,TOIEVShotsA,TOIEVDShots,TOIBlocks,TOIHits,TOIPenalties,TOITakeaways,TOIGiveaways,TOIMisses
0,2017,TOR,PATRICK MARLEAU,W,82,27.0,20.0,47.0,81.0,59.0,61.0,55.0,6.0,177.0,749,730,19,607,703,-96,26.0,85.0,16.0,28.0,33.0,67.0,1393.0,0.019383,0.014358,0.033740,0.058148,0.042355,0.043790,0.039483,0.004307,0.127064,0.537688,0.524049,0.013640,0.435750,0.504666,-0.068916,0.018665,0.061019,0.011486,0.020101,0.023690,0.048098
1,2017,TOR,NAZEM KADRI,C,80,32.0,23.0,55.0,88.0,52.0,54.0,52.0,2.0,178.0,811,714,97,597,712,-115,26.0,93.0,42.0,42.0,48.0,75.0,1341.0,0.023863,0.017151,0.041014,0.065623,0.038777,0.040268,0.038777,0.001491,0.132737,0.604773,0.532438,0.072334,0.445190,0.530947,-0.085757,0.019389,0.069351,0.031320,0.031320,0.035794,0.055928
2,2017,TOR,LEO KOMAROV,C,74,7.0,12.0,19.0,42.0,52.0,35.0,35.0,0.0,107.0,508,710,-202,463,525,-62,45.0,219.0,31.0,47.0,42.0,43.0,1168.0,0.005993,0.010274,0.016267,0.035959,0.044521,0.029966,0.029966,0.000000,0.091610,0.434932,0.607877,-0.172945,0.396404,0.449486,-0.053082,0.038527,0.187500,0.026541,0.040240,0.035959,0.036815
3,2017,TOR,NIKITA ZAITSEV,D,60,5.0,8.0,13.0,60.0,69.0,59.0,51.0,8.0,49.0,556,827,-271,549,655,-106,134.0,113.0,31.0,21.0,76.0,29.0,1333.0,0.003751,0.006002,0.009752,0.045011,0.051763,0.044261,0.038260,0.006002,0.036759,0.417104,0.620405,-0.203301,0.411853,0.491373,-0.079520,0.100525,0.084771,0.023256,0.015754,0.057014,0.021755
4,2017,TOR,JAKE GARDINER,D,82,5.0,47.0,52.0,97.0,66.0,75.0,63.0,12.0,128.0,965,935,30,806,902,-96,103.0,51.0,32.0,39.0,105.0,68.0,1847.0,0.002707,0.025447,0.028154,0.052518,0.035734,0.040606,0.034109,0.006497,0.069302,0.522469,0.506226,0.016243,0.436383,0.488360,-0.051976,0.055766,0.027612,0.017325,0.021115,0.056849,0.036816


In [141]:
de.shape

(1066, 48)

In [142]:
de.to_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/stats/2017_player_stats.csv', index='False', sep=',')
#de = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/stats/2017_player_stats.csv', index='False', sep=',')